Attempt to fully vectorise environment creation.

In [1]:
import torch
import torch.nn.functional as F

In [16]:
SIZE = 12
NUM_ENVS = 4
FOOD_CHANNEL = 0
HEAD_CHANNEL = 1
BODY_CHANNEL = 2
SNAKE_LENGTH = 4

In [3]:
envs = torch.zeros((NUM_ENVS, 3, SIZE, SIZE))

In [4]:
envs[0, HEAD_CHANNEL, 5, 5] = 1
envs[1, HEAD_CHANNEL, 5, 6] = 1
envs[2, HEAD_CHANNEL, 6, 5] = 1
envs[3, HEAD_CHANNEL, 5, 6] = 1

In [5]:
envs[:, HEAD_CHANNEL, :, :]

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [6]:
snake_filters = torch.Tensor([
    [
        [4, 0, 0, 0, 0],
        [3, 0, 0, 0, 0],
        [2, 0, 0, 0, 0],
        [1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
    ],
    [
        [4, 3, 2, 1, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
    ],
    [
        [0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0],
        [2, 0, 0, 0, 0],
        [3, 0, 0, 0, 0],
        [4, 0, 0, 0, 0],
    ],
    [
        [0, 1, 2, 3, 4],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
    ],
]).unsqueeze(1).float()
snake_filters.shape

torch.Size([4, 1, 5, 5])

In [8]:
random_directions = torch.randint(4, (NUM_ENVS, ))
random_directions_onehot = torch.FloatTensor(NUM_ENVS, 4)
random_directions_onehot.zero_()
random_directions_onehot.scatter_(1, random_directions.unsqueeze(-1), 1)

tensor([[0., 0., 0., 1.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]])

In [9]:
F.conv2d(envs[:, HEAD_CHANNEL:HEAD_CHANNEL+1, :, :], snake_filters, padding=2).shape

torch.Size([4, 4, 12, 12])

In [10]:
torch.einsum('bchw,bc->bhw', [
    F.conv2d(envs[:, HEAD_CHANNEL:HEAD_CHANNEL+1, :, :], snake_filters, padding=2),
    random_directions_onehot
]).unsqueeze(1)

tensor([[[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 4., 3., 2., 1., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]],


        [[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0

In [28]:
torch.stack([
    torch.arange(NUM_ENVS),
    torch.zeros((NUM_ENVS,)).long(),
    torch.randint(1 + SNAKE_LENGTH, SIZE - (1 + SNAKE_LENGTH), size=(NUM_ENVS,)),
    torch.randint(1 + SNAKE_LENGTH, SIZE - (1 + SNAKE_LENGTH), size=(NUM_ENVS,))
]).t()

tensor([[0, 0, 5, 6],
        [1, 0, 5, 5],
        [2, 0, 5, 5],
        [3, 0, 6, 5]])

In [14]:
torch.zeros((NUM_ENVS,))

tensor([0., 0., 0., 0.])

In [27]:
torch.randint(1 + SNAKE_LENGTH, SIZE - (1 + SNAKE_LENGTH), size=(NUM_ENVS,))

tensor([6, 6, 6, 5])